## Header
This is a notebook for developing filtering capabilities based on tie point statistics in Metashape.
 - [ ] Check distribution of tie point variance
 - [ ] Check number of neighbour cameras with matches

### Import relevant libraries

In [1]:
from pathlib import Path

import Metashape
import plotly.express as px
import polars as pl

from result import Ok, Err, Result

from benthoscan.project import Document, load_document
from benthoscan.utils.log import logger

### Load project

In [2]:
document_path: Path = Path("/home/martin/data/acfr_revisits_metashape_projects_working/qd61g27j_filtered.psz")
document: Document = load_document(document_path).unwrap()
logger.info(document.path)

LoadProject: path = /home/martin/data/acfr_revisits_metashape_projects_working/qd61g27j_filtered.psz


2024-06-16 07:56:44.592 | INFO     | __main__:<module>:2 - /home/martin/data/acfr_revisits_metashape_projects_working/qd61g27j_filtered.psz


loaded project in 61.2945 sec


### Log some attributes

In [12]:
def log_chunk_data(chunk: Metashape.Chunk) -> None:
    """TODO"""
    logger.info(f" - Chunk:          {chunk.label}")
    logger.info(f" - Point cloud:    {chunk.point_cloud}")

logger.info("---------- Document ------------")
logger.info(Path(document.path).name)
for chunk in document.chunks:
    log_chunk_data(chunk)

2024-06-16 08:12:26.530 | INFO     | __main__:<module>:6 - ---------- Document ------------
2024-06-16 08:12:26.533 | INFO     | __main__:<module>:7 - qd61g27j_filtered.psz
2024-06-16 08:12:26.533 | INFO     | __main__:log_chunk_data:3 -  - Chunk:          qd61g27j_20100421_022145
2024-06-16 08:12:26.534 | INFO     | __main__:log_chunk_data:4 -  - Point cloud:    <PointCloud '25317691 points'>
2024-06-16 08:12:26.535 | INFO     | __main__:log_chunk_data:3 -  - Chunk:          qd61g27j_20110410_011202
2024-06-16 08:12:26.535 | INFO     | __main__:log_chunk_data:4 -  - Point cloud:    <PointCloud '50593200 points'>
2024-06-16 08:12:26.536 | INFO     | __main__:log_chunk_data:3 -  - Chunk:          qd61g27j_20120422_043114
2024-06-16 08:12:26.536 | INFO     | __main__:log_chunk_data:4 -  - Point cloud:    <PointCloud '52581426 points'>
2024-06-16 08:12:26.537 | INFO     | __main__:log_chunk_data:3 -  - Chunk:          qd61g27j_20130414_013620
2024-06-16 08:12:26.537 | INFO     | __main__:

### Process point cloud

In [28]:
def get_temp_cloud(chunk: Metashape.Chunk, path: Path) -> None:
    chunk.exportPointCloud(
        path=str(path), 
        source_data=PointCloudData, 
        binary=True,
        save_point_color=True, 
        save_point_normal=True, 
        save_point_intensity=True,
        save_point_classification=True, 
        save_point_confidence=True,
        save_point_return_number=True, 
        save_point_scan_angle=True,
        save_point_source_id=True, 
        save_point_timestamp=True, 
        save_point_index=True,
        raster_transform=RasterTransformNone, 
        colors_rgb_8bit=True, 
        comment='',
        save_comment=True, 
        format=PointCloudFormatNone,
        image_format=ImageFormatJPEG, 
        clip_to_boundary=True,
        block_width=1000, 
        block_height=1000, 
        split_in_blocks=False,
        save_images=False, 
        compression=True, 
        tileset_version='1.0', 
        screen_space_error=16,
        folder_depth=5,
        subdivide_task=True,
        progress
    )

• path (str) – Path to output file.
• source_data (Metashape.DataSource) – Selects between point cloud and tie points. If
not specified, uses point cloud if available.
• point_cloud (int) – Point cloud key to export.
• binary (bool) – Enables/disables binary encoding for selected format (if applicable).
• save_point_color (bool) – Enables/disables export of point color.
43
Metashape Python Reference, Release 2.1.0
• save_point_normal (bool) – Enables/disables export of point normal.
• save_point_intensity (bool) – Enables/disables export of point intensity.
• save_point_classification (bool) – Enables/disables export of point classification.
• save_point_confidence (bool) – Enables/disables export of point confidence.
• save_point_return_number (bool) – Enables/disables export of point return number.
• save_point_scan_angle (bool) – Enables/disables export of point scan angle.
• save_point_source_id (bool) – Enables/disables export of point source ID.
• save_point_timestamp (bool) – Enables/disables export of point timestamp.
• save_point_index (bool) – Enables/disables export of point row and column indices.
• raster_transform (Metashape.RasterTransformType) – Raster band transforma-
tion.
• colors_rgb_8bit (bool) – Convert colors to 8 bit RGB.
• comment (str) – Optional comment (if supported by selected format).
• save_comment (bool) – Enable comment export.
• format (Metashape.PointCloudFormat) – Export format.
• image_format (Metashape.ImageFormat) – Image data format.
• crs (Metashape.CoordinateSystem) – Output coordinate system.
• shift (Metashape.Vector) – Optional shift to be applied to point coordinates.
• region (Metashape.BBox) – Region to be exported.
• clip_to_boundary (bool) – Clip point cloud to boundary shapes.
• block_width (float) – Block width in meters.
• block_height (float) – Block height in meters.
• split_in_blocks (bool) – Enable tiled export.
• classes (list[int]) – List of point classes to be exported.
• save_images (bool) – Enable image export.
• compression (bool) – Enable compression (Cesium format only).
• tileset_version (str) – Cesium 3D Tiles format version to export (1.0 or 1.1).
• screen_space_error (float) – Target screen space error (Cesium format only).
• folder_depth (int) – Tileset subdivision depth (Cesium format only).
• viewpoint (Metashape.Viewpoint) – Default view.
• subdivide_task:  (bool) – Enable fine-level task subdivision,
    progress (Callable[[float], None]) – Progress callback
)

def process_point_cloud(cloud: Metashape.PointCloud) -> None:
    """TODO"""
    logger.info(f"{cloud.key}")
    logger.info(f"{cloud.point_count}")
    logger.info(f"{cloud.crs}")
    logger.info(f"{cloud.component}")
    logger.info(f"{cloud.points}")

process_point_cloud(document.chunks[0].point_cloud.copy())

tmp1 = tempfile.NamedTemporaryFile(delete=False)

DuplicateAsset: asset_type = Point cloud, asset_key = 12


2024-06-16 15:51:09.225 | INFO     | __main__:process_point_cloud:3 - 13
2024-06-16 15:51:09.226 | INFO     | __main__:process_point_cloud:4 - 25317691
2024-06-16 15:51:09.228 | INFO     | __main__:process_point_cloud:5 - None
2024-06-16 15:51:09.229 | INFO     | __main__:process_point_cloud:6 - None


AttributeError: 'Metashape.Metashape.PointCloud' object has no attribute 'points'